# Live inference webcam

In [1]:
import cv2
from ultralytics import YOLO

model = YOLO('v8n_dataset_pierga.pt')
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Errore nell'aprire la webcam")
else:
    print("🎥 Webcam attiva. Premi 'q' per uscire.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(frame, imgsz=640, conf=0.7, verbose=False)[0]
    annotated_frame = results.plot()

    # 🖨️ Log su terminale
    for box in results.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        xyxy = box.xyxy[0].tolist()
        print(f"Classe: {model.names[cls_id]}, Confidenza: {conf:.2f}, BBox: {xyxy}")

    cv2.imshow("YOLOv8 - Eyes On Falls LIVE", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


🎥 Webcam attiva. Premi 'q' per uscire.
Classe: not_fallen, Confidenza: 0.73, BBox: [235.988525390625, 94.98905944824219, 544.9149780273438, 437.43267822265625]
Classe: not_fallen, Confidenza: 0.72, BBox: [232.74542236328125, 95.60113525390625, 541.43359375, 429.9208984375]
Classe: not_fallen, Confidenza: 0.72, BBox: [227.12786865234375, 7.0985107421875, 540.283203125, 480.0]
Classe: not_fallen, Confidenza: 0.74, BBox: [234.4051513671875, 2.28070068359375, 546.1187133789062, 407.98211669921875]
Classe: not_fallen, Confidenza: 0.72, BBox: [337.605712890625, 342.9576416015625, 465.2623291015625, 479.9127197265625]
Classe: not_fallen, Confidenza: 0.73, BBox: [335.84033203125, 346.2034912109375, 464.6015625, 479.93048095703125]
Classe: fallen, Confidenza: 0.71, BBox: [253.22691345214844, 19.179443359375, 561.9161376953125, 422.937744140625]
Classe: fallen, Confidenza: 0.71, BBox: [257.0455322265625, 23.170318603515625, 567.2646484375, 426.2483825683594]
Classe: fallen, Confidenza: 0.73, BBo

In [8]:
from ultralytics import YOLO

# Carichiamo il modello YOLOv11n addestrato
model = YOLO('v8n_dataset_pierga.pt')  # qui metti il tuo path .pt

# Esporta in ONNX (dinamico o fisso, in base alle esigenze NCNN)
model.export(format='onnx', opset=12, dynamic=False, imgsz=640)

# Otterrai: best.onnx

Ultralytics 8.3.154  Python-3.11.9 torch-2.7.1+cpu CPU (Intel Core(TM) i7-8565U 1.80GHz)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'v8n_dataset_pierga.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (17.6 MB)

ONNX: starting export with onnx 1.17.0 opset 12...
ONNX: slimming with onnxslim 0.1.57...
ONNX: export success  2.7s, saved as 'v8n_dataset_pierga.onnx' (11.7 MB)

Export complete (3.4s)
Results saved to C:\Users\kikop\Desktop\EOF_YOLO\Eyes_on_Falls_YOLO\2. Pierga_eof_YOLO\v8nfirst
Predict:         yolo predict task=detect model=v8n_dataset_pierga.onnx imgsz=640  
Validate:        yolo val task=detect model=v8n_dataset_pierga.onnx imgsz=640 data=data.yaml  
Visualize:       https://netron.app


'v8n_dataset_pierga.onnx'

In [5]:
import cv2
import numpy as np
import tflite_runtime.interpreter as tflite
import time

# Configurazione
MODEL_PATH = "v8n_dataset_pierga_float16.tflite"
INPUT_SIZE = 640
CONF_THRESHOLD = 0.4
IOU_THRESHOLD = 0.45
CLASS_NAMES = ["fallen", "not_fallen"]

# Load model
interpreter = tflite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def xywh2xyxy(x):
    y = np.copy(x)
    y[..., 0] = x[..., 0] - x[..., 2] / 2  # x1
    y[..., 1] = x[..., 1] - x[..., 3] / 2  # y1
    y[..., 2] = x[..., 0] + x[..., 2] / 2  # x2
    y[..., 3] = x[..., 1] + x[..., 3] / 2  # y2
    return y

def process_output(output):
    output = np.squeeze(output).transpose()  # (8400, 6)
    boxes = sigmoid(output[:, 0:4])
    scores = sigmoid(output[:, 4])
    class_scores = sigmoid(output[:, 5:])
    class_ids = np.argmax(class_scores, axis=-1)
    class_conf = np.max(class_scores, axis=-1)
    final_scores = scores * class_conf
    mask = final_scores > CONF_THRESHOLD
    boxes, final_scores, class_ids = boxes[mask], final_scores[mask], class_ids[mask]
    boxes = xywh2xyxy(boxes) * INPUT_SIZE
    return boxes, final_scores, class_ids

# Video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, INPUT_SIZE)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, INPUT_SIZE)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_resized = cv2.resize(frame, (INPUT_SIZE, INPUT_SIZE))
    input_data = np.expand_dims(img_resized, axis=0).astype(np.float32) / 255.0
    interpreter.set_tensor(input_details[0]['index'], input_data)

    start = time.time()
    interpreter.invoke()
    end = time.time()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    boxes, scores, class_ids = process_output(output_data)

    scale_x = frame.shape[1] / INPUT_SIZE
    scale_y = frame.shape[0] / INPUT_SIZE

    for i in range(len(boxes)):
        x1, y1, x2, y2 = boxes[i]
        x1, y1, x2, y2 = int(x1 * scale_x), int(y1 * scale_y), int(x2 * scale_x), int(y2 * scale_y)
        label = f"{CLASS_NAMES[class_ids[i]]}: {scores[i]:.2f}"
        color = (0, 255, 0) if class_ids[i] == 0 else (0, 0, 255)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    fps = 1 / (end - start)
    cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    cv2.imshow('YOLOv8 TFLite Live', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'tflite_runtime'